# Missing and Cleaning Data

In this section we will learn how to clean data including missing values, bad values, and casting data types. Let's start by dealing with missing values. 

## Finding Missing Values

Let's load up the dataset from the last section we ended with. 

In [1]:
import pandas as pd

url = r"https://gist.githubusercontent.com/thomasnield/e17ca671e8743a1a8fdfedb13d015b44/raw/d6e3af4f311d2c762a9738f109a1cf59aa45d420/birdstrike_section1.csv"

df = pd.read_csv(url)

with pd.option_context('display.max_columns', None):
  display(df)

,INDEX_NR,OPID,OPERATOR,AIRCRAFT,AC_CLASS,AC_MASS,NUM_ENGS,TYPE_ENG,INCIDENT_DATE,INCIDENT_YEAR,INCIDENT_MONTH,TIME_OF_DAY,TIME,AIRPORT_ID,AIRPORT,STATE,RUNWAY,LOCATION,LATITUDE,LONGITUDE,HEIGHT,SPEED,DISTANCE,PHASE_OF_FLIGHT,DAMAGE_LEVEL,STR_RAD,DAM_RAD,STR_WINDSHLD,DAM_WINDSHLD,STR_NOSE,STR_PROP,DAM_PROP,STR_WING_ROT,DAM_WING_ROT,STR_FUSE,DAM_FUSE,STR_LG,DAM_LG,STR_TAIL,DAM_TAIL,STR_LGHTS,DAM_LGHTS,STR_OTHER,EFFECT,SKY,PRECIPITATION,SPECIES_ID,SPECIES,SIZE,WARNED,COST_REPAIRS,COST_OTHER,COST_REPAIRS_INFL_ADJ,COST_OTHER_INFL_ADJ,NR_INJURIES,NR_FATALITIES,INDICATED_DAMAGE
0,708307,BUS,BUSINESS,PA-28,A,1.0,1.0,A,2015-05-22,2015,5,NaN,NaN,KVRB,VERO BEACH MUNICIPAL,FL,4,NaN,27.65556,-80.41794,NaN,NaN,NaN,Approach,M,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1
1,708308,BUS,BUSINESS,BE-1900,A,3.0,2.0,C,2015-06-18,2015,6,NaN,NaN,PAEN,KENAI MUNICIPAL ARPT,AK,NaN,NaN,60.57200,-151.24753,NaN,NaN,NaN,Approach,M,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,NaN,NaN,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1
2,708309,BUS,BUSINESS,PA-46 MALIBU,A,1.0,1.0,A,2015-09-20,2015,9,NaN,NaN,KDWH,DAVID WAYNE HOOKS MEMORIAL ARPT,TX,NaN,NaN,30.06186,-95.55278,NaN,NaN,NaN,NaN,M,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1
3,708310,DAL,DELTA AIR LINES,B-717-200,A,4.0,2.0,D,2015-11-07,2015,11,NaN,NaN,KSTL,LAMBERT-ST LOUIS INTL,MO,30R,NaN,38.74769,-90.35999,NaN,NaN,2.0,Approach,M,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,NaN,NaN,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1
4,708311,BUS,BUSINESS,BE-90 KING,A,2.0,2.0,C,2015-12-17,2015,12,NaN,NaN,KPMP,POMPANO BEACH AIRPARK,FL,15,NaN,26.24714,-80.11106,0.0,NaN,0.0,Landing Roll,M,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,NaN,NaN,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,765256,AWE,US AIRWAYS,A-320,A,4.0,2.0,D,2015-01-08,2015,1,Day,14:50,KPHX,PHOENIX SKY HARBOR INTL ARPT,AZ,NaN,NaN,33.43417,-112.00806,NaN,NaN,NaN,Approach,N,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Some Cloud,NaN,UNKB,Unknown bird,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0
913,765257,AAL,AMERICAN AIRLINES,B-737-800,A,4.0,2.0,D,2015-01-09,2015,1,Day,16:00,KJFK,JOHN F KENNEDY INTL,NY,31R,NaN,40.63975,-73.77893,0.0,NaN,0.0,Landing Roll,N,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,NaN,Some Cloud,NaN,YH004,Horned lark,Small,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0
914,765258,AAL,AMERICAN AIRLINES,MD-83,A,4.0,2.0,D,2015-01-09,2015,1,Day,12:40,KDFW,DALLAS/FORT WORTH INTL ARPT,TX,35L,NaN,32.89595,-97.03720,0.0,NaN,0.0,Take-off Run,N,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,Overcast,NaN,ZT000,Meadowlarks,Small,Yes,NaN,NaN,NaN,NaN,NaN,NaN,0
915,765259,BUS,BUSINESS,CITATIONJET,A,2.0,2.0,D,2015-01-09,2015,1,Night,05:40,KSMO,SANTA MONICA MUNICIPAL,CA,21,NaN,34.01582,-118.45130,0.0,100.0,0.0,Take-off Run,N,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,No Cloud,NaN,UNKBS,Unknown bird - small,Small,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0


The `LATITUDE` and `LONGITUDE` columns have caused some warnings: ` DtypeWarning: Columns (18,19) have mixed types.`. We will address this later. 

Notice how there are `NaN` and `None` values in the dataframe. These are missing values, and these gaps in the data happen for various reasons often specific to the domain. Sometimes these missing values are a showstopping problem, such as weather sensors being broken and needing to be addressed. For some other projects, we can simply remove or guess their values using imputation without causing too many problems. 

But first, we need to see which parts of the data have holes and how big those holes are. We do this by quantifying our missing values. Missing values can show up as `NaN` or `None`, depending on whether the missing value is numeric or not. The `isnull()` function will identify both.

Let's first remind ourselves the number of rows and columns we have. 

In [ ]:
df.shape

Then let's call `isnull()` which will generate `True` and `False` values depending if the value is missing. Then we call `sum()` to count the number of `True` values (as `True` will be treated as `1` and `False` will be treated as `0`). 

In [ ]:
df.isnull().sum()

It looks like a lot of these fields are missing values. Let's calculate the percentage of missing values for each field. 

In [ ]:
missing_pct = pd.DataFrame({'Missing Values': df.isnull().sum(),
                              'Percentage': df.isnull().sum() / df.shape[0]})

missing_pct.sort_values(by='Percentage', ascending = False, inplace=True)

missing_pct

So what do we do here? Do we just throw away certain columns because they have a high proportion of missing values? It depends. `NR_INJURIES` and `NR_FATALITIES` have the highest amount of missing values, but perhaps they are simply blank because there were no injuries or fatalities on those bird strikes. We definitely don't want to exclude those. Some fields are a little bit more suspicous for their missing values, like `PHASE_OF_FLIGHT`. Let's not remove anything for now because a lot of how we handle missing values will depend on the task. Let's look at the values next and figure out their levels of measurement. 

## Categorical and Numerical Variables

How we interpret variables depends on whether they are nominal, ordinal, interval, or ratio. 

| Level of Measurement | Description                                                                                  |
|----------------------|----------------------------------------------------------------------------------------------|
| Nominal              | Categories without any ordering (e.g. dog, cat, bird)                                        |
| Ordinal              | Has an inherent order or ranking with unequal intervals (e.g. a video game score)            |
| Interval             | Have a meaningful order with consistent intervals, but no zero point (e.g. a time stamp)     |
| Ratio                | Have a meaningful order, consistent intervals, and a zero point (e.g. body weight or height) |

Let's count the number of unique values in each column. We will use `nunique()` but not exclude null values by specifying `dropna=False`. 

In [ ]:
df.nunique(dropna=False).sort_values()

Everything with a 2-digit number of unique values are likely categorical. If there are 2 unique values they are likely boolean True/False values. 3 unique values might be boolean but with a null value as well. 

Let's concatenate these unique values for each column, so we can get a quick overview of what the unique values look like. 

In [ ]:
unique_val_ct = df.nunique(dropna=False).sort_values()

unique_vals = dict()
for col in unique_val_ct.index:
    unique_vals[col] = pd.Series(df[col].unique()).astype(str).str.cat(sep=',')

unique_val_show = pd.DataFrame({
    "COUNT" : unique_val_ct,
    "VALUES" : unique_vals
})

with pd.option_context('display.max_rows', None, 'display.max_colwidth', 60):
    display(unique_val_show)

### Converting Boolean Columns

Okay... lots to unpack here. Those values containing only a `0` or `1` are definitely booleans. Let's get those converted right now. 

In [ ]:
binary_cols = ['DAM_WINDSHLD', 'STR_WINDSHLD', 'STR_NOSE', 'STR_PROP', 'DAM_PROP',
       'STR_WING_ROT', 'DAM_WING_ROT', 'STR_FUSE', 'DAM_FUSE', 'STR_LG',
       'DAM_LG', 'STR_TAIL', 'DAM_TAIL', 'STR_LGHTS', 'DAM_LGHTS',
       'STR_OTHER', 'DAM_RAD', 'STR_RAD', 'INDICATED_DAMAGE']

for col in binary_cols: 
    df[col] = df[col].astype('bool')

with pd.option_context('display.max_columns', None):
  display(df)

Now the binary columns have been converted. The `WARNED` column is interesting because it too is a binary with `Yes` and `No`, but it could also be `Unknown`. According to the documentation, this indicates whether or not the pilot was warned about the birds ahead before the strike happened. Let's get a count of those values.

In [ ]:
df["WARNED"].value_counts()

So there are a lot of unknowns. Geez, if we only had data on the avoided bird strikes as a result of the warning, this column could be a lot more useful to see if the warnings had any effect. Unfortunately, we are only getting reports when the bird strike happened so there's some survivor bias going on here to ask such a question. 

Let's just make the `WARNED` column a `boolean` type (which allows `None` values unlike the `bool` type which will treat them as `False`). 

In [ ]:
df["WARNED"] = df["WARNED"].map({"Yes": True, "No": False, "Unknown": pd.NA}).astype('boolean')

df["WARNED"].value_counts(dropna=False)

## Converting Dates and Times

The `INCIDENT_DATE` is the date (in local time) the incident occurred. We can easily convert a column of date strings that carry a typical format using Pandas' `to_datetime()`. 

In [ ]:
pd.to_datetime(df['INCIDENT_DATE'])

Now before we assign that back to the dataframe, let's note the `TIME` column exists as well. This is the time the incident occured in local time. It is tempting to concatenate the two columns `INCIDENT_DATE` and `TIME` together and turn it into a single `datetime`, but there are `nan` values which is unfortunate. We cannot merge the two string columns together when one of them as nulls. 

What we will do is convert the `TIME` column to timedelta types rather than a string. However, to make our data cleaning a bit more annoying, I was having errors. Apparently there are values that are simply empty whitespace and not nulls. 😡

How did I figure this out? Well, first I had to use a regular expression in a `contains()` function ([which Anaconda also covers in a course](https://learning.anaconda.cloud/regular-expressions-in-python)) to filter out strings that do not match the hour:minute format. Then I took a `value_count()` of these deviant values. 

In [ ]:
not_time_format= ~df["TIME"].fillna(value="", inplace=False).str.contains(r"[0-9]{1,2}[:][0-9]{2}")

df["TIME"][not_time_format].value_counts(dropna=False)

Now that I know what I am dealing with, I can use another regular expression in a `replace()` function to change the blank spaces into `nan` values. Then I can append a `:00` for the seconds, and finally convert the `TIME` column to a `timedelta`. Note that the `nan`'s are going to be converted into `NaT`'s which are timedelta's blank values.

In [ ]:
import numpy as np 

# remove blank spaces and change to nan 
df["TIME"] = df["TIME"].replace(r'^\s*$', np.nan, regex=True)

# append seconds 
df["TIME"] += ':00'

# convert to timedelta 
df["TIME"] = pd.to_timedelta(df["TIME"])
df["TIME"]

Anaconda also has a course [Data Cleaning with Pandas](https://learning.anaconda.cloud/data-cleaning-with-pandas) that has a dedicated module on date and time conversion. 

## Converting Numeric Data 

When I tried to convert the `LATITUDE` and `LONGITUDE` columns to floating point values rather than strings, something funky was happening. I was getting errors I'm going to use another regular expression to find things that do not match the expected numeric pattern in the strings. 

In [ ]:
bad_latitudes = ~df["LATITUDE"].fillna(value="", inplace=False).str.contains(r"^-?[0-9]+\.[0-9]+$").astype(bool)

df[bad_latitudes]["LATITUDE"].value_counts(dropna=False)

There are a lot of missing values, fine. But there are four weirdly formatted or broken values that are unable to be converted to floating point values. Let's look at longitude next.

In [ ]:
bad_longitudes = ~df["LONGITUDE"].fillna(value="", inplace=False).str.contains(r"^-?[0-9]+\.[0-9]+$").astype(bool)

df[bad_longitudes]["LONGITUDE"].value_counts(dropna=False)

Similar situation. Lots of missing values, but one weirdly formatted longitude value. Let's just turn these into blank values. 

In [ ]:
df.loc[bad_longitudes, "LONGITUDE"] = np.nan 
df.loc[bad_latitudes, "LATITUDE"] = np.nan 

Finally, now that we have those weird values removed we can now convert these to floating point values without errors. 

In [ ]:
df["LATITUDE"] = df["LATITUDE"].astype(float)
df["LONGITUDE"] = df["LONGITUDE"].astype(float)

## Converting Categorical Data 

At times, there are going to be columns in a dataframe that only allow a few values. When these values are strings, it becomes all the more important to consider converting them into a category type. Behind the scenes, this will improve the performance of the dataframe and eliminate redundancy due to duplicate strings. 

There are lots of columns in this dataset that seem to be categorical. Let's focus on `PHASE_OF_FLIGHT` for now. This is the part of the flight when the bird strike occurred. Let's see these possible values.

In [ ]:
df["PHASE_OF_FLIGHT"].unique()

Okay there are 12 phases of flight plus the missing `nan` values. But something is awkward here about the `Unknown` category, as we have to ask what differentiates that from a `nan`? Should we turn `Unknown` into `NaN`? Let's take a look at the value counts. 

In [ ]:
df["PHASE_OF_FLIGHT"].value_counts(dropna=False)

Okay, so there are only two records where the `PHASE_OF_FLIGHT` is actually "Unkown" and 63,261 are `NaN`. Because there are so few records it is not as urgent to address them. They are certainly outliers. 

Let's look at those two records for curiosity's sake. 

In [ ]:
df[df["PHASE_OF_FLIGHT"] == "Unknown"]

Huh, weird. We got a gulls strike on that first record so that's some information if we are interested in the `SPECIES`. The experimental privately owned aircraft with an unknown bird is all mystery. Let's just keep both of them but we may remove them for certain tasks. We will turn those two values into `NaN` though. 

In [ ]:
df.loc[df["PHASE_OF_FLIGHT"] == "Unknown", "PHASE_OF_FLIGHT"] = np.nan 

Now here is something helpful we can do. We can create a custom category type that will efficiently turn these strings into a set of categories. We just declare a `CategoricalDType` and specify the categories as a list of strings. We then cast that column to that custom category type through `astype()`. 

In [ ]:
phase_of_flt = pd.CategoricalDtype(categories=['Parked', 'Taxi','Take-off Run', 'Approach', 'Departure', 'Climb', 'En Route',
                                               'Descent', 'Landing Roll', 'Arrival', 'Local'])

df["PHASE_OF_FLIGHT"] = df["PHASE_OF_FLIGHT"].astype(phase_of_flt)

Now we can confirm the datatype is indeed this category we created. 

In [ ]:
df["PHASE_OF_FLIGHT"].dtype

What also might be helpful is these categories are sortable based on the ordering you defined them. The categories were specified in an order that follows the sequence of stages in a flight (e.g. `Parked` happens before `Taxi`, and `Taxi` happens before `Take-off Run`, etc). When we sort those values we will see this reflected. 

In [ ]:
df["PHASE_OF_FLIGHT"].unique().sort_values()

Let's save our work to a CSV for the next section. Note though the categories are going to be saved as strings for simplicity's sake. But we can always convert them to categories again after ingestion to keep the `DataFrame` more efficient.  

In [ ]:
df.to_csv('birdstrike_section2.csv')

## Exercise 

Get the value counts for `AIRPORT_ID` values that do not start with the letter `K`. The conditional boolean array has already been completed for you. Just provide the code in the question mark `?` 

In [ ]:
condition = ~df["AIRPORT_ID"].fillna(value="", inplace=False).str.contains(r"^K")

not_k_airports = ?

with pd.option_context('display.max_rows', None):
    display(not_k_airports)

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
condition = ~df["AIRPORT_ID"].fillna(value="", inplace=False).str.contains(r"^K")

not_k_airports = df[condition]["AIRPORT_ID"].value_counts()

with pd.option_context('display.max_rows', None):
    display(not_k_airports)